<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/wordmark_black65ee464.png" width="700">

# Week 2 : Final Challenge

**Welcome to the final challenge!**  

In the previous notebook we've seen how we can use the VQC class in Aqua to classify the digits `0` and `1`. However, classifying `0` and `1` is relatively simple as digits `0` and `1` are easily distinguishable. `4` and `9` however, are notoriously similar, with a _loop_ on the top and a _line_ on the bottom. This can be corroborated looking at our 2-D t-SNE plot from the previous notebook (Fig.2), we see that `0` and `1` are clustered relatively far from each other making them easily distinguishable, however `4` and `9` are overlapping. In this challenge we are providing you with a dataset with digits reduced to **dimension 3**. For example, in Fig.1 we can see the dimension reduction of the 784 dimension vector for digit `4` into a dimension 3 feature vector. 

**Fig.1 : Features of the digit `4` after reducing dimension to 3:** 
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/four2a7701f.png" width="700">

**Fig.2 : MNIST dataset after dimension reduction to 2 as given in the previous notebook:**
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/mnist_plot53adb39.png" width="400">

## Challenge Question   
Use the VQC method from Aqua to classify the digits `4` and `9` as given in the dataset **challenge_dataset_4_9_dim3_training.csv** provided to you. 

## Rules and Guidelines

* Your `QuantumCircuit` can have a **maximum of 6 qubits**.
* **Cost of the circuit should be less than 2000**.  
* You should not change names of the functions `feature_map()` , `variational_circuit()`  and `return_optimal_params()`.
* All the functions must return the value types as mentioned. 
* All circuits must be Qiskit generated.
* Best of all submissions is considered for grading.

## Judging criteria 

* Primary judgement is based on the **accuracy of the model**, higher the better. **Accuracies which differ by less than 0.005 will be considered to be equal**. ex: Accuracies 0.7783 and 0.7741 will be considered to be equal.
* If the accuracies are tied, the tie will be broken using **cost of the circuit** as the metric, lower the better. 
* In the case that both accuracy of the model and cost of the circuit are equal, **time of submission** is taken into account, Earlier the better. 

_**Important Note:**_ The **leaderboard shown during the progress of the competition** will only display accuracy of the model and is **not the final leaderboard**. Breaking ties between accuracy of the model by considering lower **cost of circuit** will only be done after the competition ends. **The final leaderboard will be announced post the event** which will take into consideration cost of the circuit and time of submission. 

## Certificate Eligibility

Everyone who scores an **accuracy greater than 0.70 (i.e, 70%) will be eligible for a certificate**. 


An explanation on how to calculate the accuracy of the model and the cost of the circuit is given in the end inside the `grade()` function. Before you submit, make sure the grading function is running on your device. To save time you can also use the grading function provided to calculate the accuracy and circuit cost without having to submit your solution onto HackerEarth. Remember, your final score will be determined using the same grading methods as given in this notebook, but will be evaluated on unseen datapoints.

In [ ]:
# installing a few dependencies
!pip install --upgrade seaborn==0.10.1
!pip install --upgrade scikit-learn==0.23.1
!pip install --upgrade matplotlib==3.2.0
!pip install --upgrade pandas==1.0.4
!pip install --upgrade qiskit==0.19.6 
!pip install --upgrade plotly==4.9.0

# the output will be cleared after installation
from IPython.display import clear_output
clear_output()

Requirement already up-to-date: seaborn==0.10.1 in c:\users\codie\appdata\local\programs\python\python37\lib\site-packages (0.10.1)


You should consider upgrading via the 'c:\users\codie\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [ ]:
# we have imported a few libraries we thing might be useful 
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer

from qiskit import *
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram
%matplotlib inline
import matplotlib.pyplot as plt

import time
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap, RealAmplitudes, EfficientSU2,TwoLocal
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import COBYLA


# The the write_and_run() magic function creates a file with the content inside the cell that it is run. 
# You have used this in previous exercises for creating your submission files. 
# It will be used for the same purpose here.

from IPython.core.magic import register_cell_magic
@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

# Solution

## Data loading 

This notebook has helper functions and code snippets to save your time and help you concentrate on what's important: Increasing the accuracy of your model. Running the cell below will import the challenge dataset and will be available to you as `data`. Before running the cell below store the dataset in this file structure (or change the `data_path` accordingly):  

- `challenge_notebook.ipynb`
- `dataset`
    - `challenge_dataset_4_9.csv`


In [ ]:
#data_path='./dataset/'
data = np.loadtxt( "challenge_dataset_4_9_dim3_training.csv", delimiter=",")

# extracting the first column which contains the labels
data_labels = data[:, :1].reshape(data.shape[0],)
# extracting all the columns but the first which are our features
data_features = data[:, 1:]

## Visualizing the dataset

Before we dive into solving the question it is always beneficial to look at the dataset pictographically. This will help us understand patterns which we could leverage when designing our feature maps and variational circuits for example.

In [ ]:
import plotly.express as px
import pandas as pd

# creating a dataframe using pandas only for the purpose fo plotting
df = pd.DataFrame({'Component 0':data_features[:,0], 'Component 1':data_features[:,1], 
                   'Component 2':data_features[:,2], 'label':data_labels})

fig = px.scatter_3d(df, x='Component 0', y='Component 1', z='Component 2', color='label')
fig.show()

## Extracting the training dataset

The given dataset has already been reduced in dimension and normalized, so, further pre-processing isn't techincally required. You can do so if you want to, but the testing dataset will be of the same dimension and normalisation as the training dataset provided. Training a dataset of size 6,000 will take multiple hours so you'll need to extract a subset of the dataset to use as a training dataset. The accuracy of the model may vary based on the datapoints and size of the training dataset you choose. Thus, experimenting with various sizes and datapoints will be necessary. For example, Increasing the training dataset size may increase the accuracy of the model however it will increase the training time as well.

Use the space below to extract your training dataset from `data`. For your convenience `data` has been segregated into `data_labels` and `data_features`.

* `data_labels` : 6,000 $\times$ 1 column vector with each entry either `4` or `9` 
* `data_features` : 6,000 $\times$ 3 matrix with each row having the feature corresponding to the label in `data_labels`

**Note:** This process was done in the previous [VQC notebook](https://github.com/Qiskit-Challenge-India/2020/blob/master/Day%206%2C%207%2C8/VQC_notebook.ipynb) with `0` and `1` labels and can be modified and used here as well. 

In [ ]:
### WRITE YOUR CODE BETWEEN THESE LINES - START

# do your classical pre-processing here
 #### --------1) seperate data for 9 and 4
four_data_set = [] 
nine_data_set = []

for i in range(1,6000):
    if data_labels[i] == 4:                   # extracting zeros
        four_data_set.append(data_features[i])

for j in range(1, 6000):
    if data_labels[j] == 9:                   # extracting ones
        nine_data_set.append(data_features[j])
               
four_data_set= np.array(four_data_set) # just made it to be an numpy array 
nine_data_set= np.array(nine_data_set)

###------2) train and test arrays
train_size = 2000
test_size = 600
val = 600
#four_train = []
#nine_train = []
#for i in range (train_size):
#    four_train.append((four_data_set[2*i]+four_data_set[2*i+1])/2)
#    nine_train.append((nine_data_set[2*i]+nine_data_set[2*i+1])/2)
four_train = four_data_set[:train_size]
nine_train = nine_data_set[:train_size]

four_test = four_data_set[-test_size:]
nine_test = nine_data_set[-test_size:]

four_val = four_data_set[train_size:train_size+val]
nine_val = nine_data_set[train_size:train_size+val]

# store your training and testing datasets to be input in the VQC optimizer in the "training_input" and 
# "testing_input" variables respectively. These variables will eb accessed whiile creating a VQC instance later. 
training_input = None
testing_input = None

training_input = {'4':four_train, '9':nine_train}
test_input = {'4':four_test, '9':nine_test}
val_input = np.concatenate((four_val,nine_val))


In [ ]:
print(len(four_test))

## Building a Quantum Feature Map

Given below is the `feature_map()` function. It takes no input and has to return a feature map which is either a `FeatureMap` or `QuantumCircuit` object. In the previous notebook you've learnt how feature maps work and the process of using existing feature maps in Qiskit or creating your own. In the space given **inside the function** you have to create a feature map and return it.   


**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `feature_map.py`

In [ ]:
"""
%%write_and_run feature_map.py
# the write_and_run function writes the content in this cell into the file "feature_map.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap

    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def feature_map(): 
    # BUILD FEATURE MAP HERE - START
    
    # import required qiskit libraries if additional libraries are required
    
   
    # build the feature map
    
    
    feature_dim = 3     # equal to the dimension of the data
    #custom_circ = QuantumCircuit(6)
    #custom_circ.h(0)
    #custom_circ.append(ZZFeatureMap(feature_dimension=feature_dim, reps=1),[1,2,4])
    #custom_circ.append(ZFeatureMap(feature_dimension=feature_dim, reps=1),[3,4,5])
    feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=3)
    #feature_map = RawFeatureVector(feature_dimension=feature_dim)
    #feature_map = PauliFeatureMap(feature_dimension=feature_dim, reps=1, paulis = ['Z','X','YY'])
    #feature_map_2 = ZZFeatureMap(feature_dimension=feature_dim, reps=1)
    #feature_map = feature_map_1.append(feature_map_2)
    #custom_circ.h(3)
    #custom_circ.h(5)
    
    
    # BUILD FEATURE MAP HERE - END
    
    #return the feature map which is either a FeatureMap or QuantumCircuit object
    return feature_map
    """

In [ ]:
%%write_and_run feature_map.py
from qiskit.aqua.components.feature_maps import FeatureMap
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import BlueprintCircuit
import numpy as np
import matplotlib.pyplot as plt
import functools

from qiskit import BasicAer
from qiskit.circuit.library import ZFeatureMap,ZZFeatureMap, PauliFeatureMap
from qiskit.aqua import QuantumInstance
from qiskit.aqua.components.feature_maps import self_product
from qiskit.aqua.algorithms import QSVM
from qiskit.ml.datasets import ad_hoc_data
from numpy import pi
class CustomFeatureMap(FeatureMap):
    """Mapping data with a custom feature map."""
    
    def __init__(self, feature_dimension, depth=2, entangler_map=None):
        """
        Args:
            feature_dimension (int): number of features
            depth (int): the number of repeated circuits
            entangler_map (list[list]): describe the connectivity of qubits, each list describes
                                        [source, target], or None for full entanglement.
                                        Note that the order is the list is the order of
                                        applying the two-qubit gate.        
        """
        self._support_parameterized_circuit = False
        self._feature_dimension = feature_dimension
        self._num_qubits = self._feature_dimension = feature_dimension+1
        self._depth = depth
        self._entangler_map = None
        if self._entangler_map is None:
            self._entangler_map = [[i, j] for i in range(self._feature_dimension) for j in range(i + 1, self._feature_dimension)]
            
    def construct_circuit(self, x, qr, inverse=False):
        """Construct the feature map circuit.
        
        Args:
            x (numpy.ndarray): 1-D to-be-transformed data.
            qr (QauntumRegister): the QuantumRegister object for the circuit.
            inverse (bool): whether or not to invert the circuit.
            
        Returns:
            QuantumCircuit: a quantum circuit transforming data x.
        """
        qc1 = QuantumCircuit(4)

        
        for _ in range(self._depth):
            y = -1.3*x[0]+x[1]
            qc1.h(0)
            qc1.h(1)
            qc1.h(2)
            qc1.h(3)
            qc1.u1(x[0],0)
            qc1.u1(x[1],1)
            qc1.u1(x[2],2)
            qc1.u1(y,3)
            qc1.cx(0,1)
            qc1.u1((2*(pi-x[0])*(pi-x[1])),1)
            qc1.cx(0,1)
            qc1.cx(0,2)
            qc1.u1((2*(pi-x[0])*(pi-x[2])),2)
            qc1.cx(0,2)
            qc1.cx(0,3)
            qc1.u1((2*(pi-x[0])*(pi-y)),3)
            qc1.cx(0,3)
            qc1.cx(1,2)
            qc1.u1((2*(pi-x[1])*(pi-x[2])),2)
            qc1.cx(1,2)
            qc1.cx(1,3)
            qc1.u1((2*(pi-x[1])*(pi-y)),3)
            qc1.cx(1,3)
            qc1.cx(2,3)
            qc1.u1((2*(pi-x[2])*(pi-y)),3)
            qc1.cx(2,3)

            
            
            
        if inverse:
            qc1.inverse()
        return qc1
def feature_map():
    return CustomFeatureMap(feature_dimension=3, depth=2)

In [ ]:
#feature_map().draw()

## Building a Variational Circuit

Given below is the `variational_circuit()` function. It takes no input and has to return a variational circuit which is either a `VariationalForm` or `QuantumCircuit` object. In the previous notebook you've learnt how variational circuits work and the process of using existing variational circuit in Qiskit or creating your own. You have to create a variational circuit in the space given **inside the function** and return it. You can find various variational circuits in the [Qiskit Circuit Library](https://qiskit.org/documentation/apidoc/circuit_library.html) under N-local circuits.

**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `variational_circuit.py`

In [ ]:
%%write_and_run variational_circuit.py
# the write_and_run function writes the content in this cell into the file "variational_circuit.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import  RealAmplitudes, EfficientSU2
    
### WRITE YOUR CODE BETWEEN THESE LINES - END
import math

def nCr(n,r):
    f = math.factorial
    return int(f(n) / f(r) / f(n-r))

def variational_circuit():
    # BUILD VARIATIONAL CIRCUIT HERE - START
    
    # import required qiskit libraries if additional libraries are required
    
    # build the variational circuit
    #var_circuit = EfficientSU2(num_qubits=3, su2_gates= ['rx', 'ry'], entanglement='circular', reps=3)
    #var_circuit = EfficientSU2(num_qubits=4, su2_gates= ['rx', 'ry'], entanglement='circular', reps=3)
    
    # BUILD VARIATIONAL CIRCUIT HERE - END
    
    # return the variational circuit which is either a VaritionalForm or QuantumCircuit object
    from qiskit.circuit import QuantumCircuit, ParameterVector

    num_qubits = 3            
    reps = 1              # number of times you'd want to repeat the circuit

    x = ParameterVector('x', length=num_qubits)  # creating a list of Parameters
    custom_circ = QuantumCircuit(num_qubits)

    # defining our parametric form
    for _ in range(reps):
        for i in range(num_qubits):
            custom_circ.rx(x[i], i)
        for i in range(num_qubits):
            for j in range(i + 1, num_qubits):
                custom_circ.cx(i, j)
                custom_circ.u1(x[i] * x[j], j)
                custom_circ.cx(i, j)
            
    custom_circ.draw()
        
            
    
                
            
    #custom_circ.draw()
    return custom_circ

## Choosing a Classical Optimizer

In the `classical_optimizer()` function given below you will have to import the optimizer of your choice from [`qiskit.aqua.optimizers`](https://qiskit.org/documentation/apidoc/qiskit.aqua.components.optimizers.html) and return it. This function will not be called by the grading function `grade()` and thus the name of the function `classical_optimizer()`can be changed if needed. 

In [29]:
def classical_optimizer():
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - START
    
    # import the required clasical optimizer from qiskit.aqua.optimizers
    from qiskit.aqua.components.optimizers import COBYLA

    cls_opt = COBYLA(maxiter=500, tol=0.001)
    
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - END
    return cls_opt

### Callback Function

The `VQC` class can take in a callback function to which the following parameters will be passed after every optimization cycle of the algorithm:

* `eval_count` : the evaulation counter
* `var_params` : value of parameters of the variational circuit
* `eval_val`  : current cross entropy cost 
* `index` : the batch index

In [30]:
def call_back_vqc(eval_count, var_params, eval_val, index):
    print("eval_count: {}".format(eval_count))
    print("var_params: {}".format(var_params))
    print("eval_val: {}".format(eval_val))
    print("index: {}".format(index))

## Optimization Step

This is where the whole VQC algorithm will come together. First we create an instance of the `VQC` class. 

In [31]:
# a fixed seed so that we get the same answer when the same input is given. 
seed = 10598

# setting our backend to qasm_simulator with the "statevector" method on. This particular setup is given as it was 
# found to perform better than most. Feel free to play around with different backend options.
backend = Aer.get_backend('qasm_simulator')
backend_options = {"method": "statevector"}

# creating a quantum instance using the backend and backend options taken before
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed, 
                                   backend_options=backend_options)

# creating a VQC instance which you will be used for training. Make sure you input the correct training_dataset and 
# testing_dataset as defined in your program.
vqc = VQC(optimizer=classical_optimizer(), 
          feature_map=feature_map(), 
          var_form=variational_circuit(), 
          callback=call_back_vqc, 
          training_dataset=training_input,     # training_input must be initialized with your training dataset
          test_dataset=test_input,datapoints = val_input)             # testing_input must be initialized with your testing dataset

c:\users\codie\appdata\local\programs\python\python37\lib\site-packages\qiskit\aqua\algorithms\classifiers\vqc.py:146: DeprecationWarning:

The qiskit.aqua.components.feature_maps.FeatureMap object is deprecated as of 0.7.0 and will be removed no earlier than 3 months after the release. You should pass a QuantumCircuit object instead. See also qiskit.circuit.library.data_preparation for a collection of suitable circuits.



Now, let's run the VQC classification routine

In [ ]:
start = time.process_time()

result = vqc.run(quantum_instance)

print("time taken: ")
print(time.process_time() - start)

print("testing success ratio: {}".format(result['testing_accuracy']))

eval_count: 1
var_params: [ 3.78831481e+00  1.88673295e-01 -1.14580183e-01  9.59212970e-01
 -2.76590574e-01  6.78948149e-01  2.25287943e-01  3.45571833e-01
  1.13617143e+00  1.05384967e+00  3.13445077e-01 -1.12628313e+00
  9.84171275e-01 -3.32930113e-03  2.23113782e-01 -4.49927914e-01
 -4.58457910e-01 -1.11324333e+00 -2.28069307e+00 -1.27950966e+00
 -7.67893290e-01 -1.11998909e+00  1.00749096e+00  1.04389855e+00
 -6.54513090e-01  5.73675145e-03  6.95097752e-01 -8.13998679e-01
 -6.60091730e-01 -2.09112228e+00 -1.76516552e-01 -5.04395360e-01
  4.97208300e-01  2.58272949e-01 -8.52477779e-01 -2.36676150e-01
  1.38002264e+00 -3.76375148e-01]
eval_val: 0.6729708331119872
index: 1
eval_count: 2
var_params: [ 3.78831481e+00  1.18867330e+00 -1.14580183e-01  9.59212970e-01
 -2.76590574e-01  6.78948149e-01  2.25287943e-01  3.45571833e-01
  1.13617143e+00  1.05384967e+00  3.13445077e-01 -1.12628313e+00
  9.84171275e-01 -3.32930113e-03  2.23113782e-01 -4.49927914e-01
 -4.58457910e-01 -1.11324333e+0

eval_count: 13
var_params: [ 3.78831481e+00  1.18867330e+00 -1.14580183e-01  9.59212970e-01
  7.23409426e-01  6.78948149e-01  2.25287943e-01  1.34557183e+00
  2.13617143e+00  2.05384967e+00  3.13445077e-01 -1.12628313e+00
  1.98417128e+00 -3.32930113e-03  2.23113782e-01 -4.49927914e-01
 -4.58457910e-01 -1.11324333e+00 -2.28069307e+00 -1.27950966e+00
 -7.67893290e-01 -1.11998909e+00  1.00749096e+00  1.04389855e+00
 -6.54513090e-01  5.73675145e-03  6.95097752e-01 -8.13998679e-01
 -6.60091730e-01 -2.09112228e+00 -1.76516552e-01 -5.04395360e-01
  4.97208300e-01  2.58272949e-01 -8.52477779e-01 -2.36676150e-01
  1.38002264e+00 -3.76375148e-01]
eval_val: 0.5927359858987689
index: 13
eval_count: 14
var_params: [ 3.78831481  1.1886733  -0.11458018  0.95921297  0.72340943  0.67894815
  0.22528794  1.34557183  2.13617143  2.05384967  0.31344508 -1.12628313
  0.98417128  0.9966707   0.22311378 -0.44992791 -0.45845791 -1.11324333
 -2.28069307 -1.27950966 -0.76789329 -1.11998909  1.00749096  1.04389

eval_count: 26
var_params: [ 3.78831481e+00  1.18867330e+00 -1.14580183e-01  9.59212970e-01
  7.23409426e-01  6.78948149e-01  2.25287943e-01  1.34557183e+00
  2.13617143e+00  2.05384967e+00  3.13445077e-01 -1.12628313e+00
  9.84171275e-01 -3.32930113e-03  1.22311378e+00 -4.49927914e-01
 -4.58457910e-01 -1.11324333e+00 -2.28069307e+00 -1.27950966e+00
 -7.67893290e-01 -1.11998909e+00  1.00749096e+00  1.04389855e+00
 -6.54513090e-01  1.00573675e+00  6.95097752e-01 -8.13998679e-01
 -6.60091730e-01 -2.09112228e+00 -1.76516552e-01 -5.04395360e-01
  4.97208300e-01  2.58272949e-01 -8.52477779e-01 -2.36676150e-01
  1.38002264e+00 -3.76375148e-01]
eval_val: 0.5952535684837749
index: 26
eval_count: 27
var_params: [ 3.78831481e+00  1.18867330e+00 -1.14580183e-01  9.59212970e-01
  7.23409426e-01  6.78948149e-01  2.25287943e-01  1.34557183e+00
  2.13617143e+00  2.05384967e+00  3.13445077e-01 -1.12628313e+00
  9.84171275e-01 -3.32930113e-03  1.22311378e+00 -4.49927914e-01
 -4.58457910e-01 -1.11324333

eval_count: 38
var_params: [ 3.78831481e+00  1.18867330e+00 -1.14580183e-01  9.59212970e-01
  7.23409426e-01  6.78948149e-01  2.25287943e-01  1.34557183e+00
  2.13617143e+00  2.05384967e+00  3.13445077e-01 -1.12628313e+00
  9.84171275e-01 -3.32930113e-03  1.22311378e+00 -4.49927914e-01
 -4.58457910e-01 -1.11324333e+00 -2.28069307e+00 -1.27950966e+00
 -7.67893290e-01 -1.11998909e+00  1.00749096e+00  1.04389855e+00
 -6.54513090e-01  5.73675145e-03  6.95097752e-01 -8.13998679e-01
 -6.60091730e-01 -2.09112228e+00 -1.76516552e-01 -5.04395360e-01
  4.97208300e-01  2.58272949e-01 -8.52477779e-01 -2.36676150e-01
  1.38002264e+00  6.23624852e-01]
eval_val: 0.7004160656024911
index: 38
eval_count: 39
var_params: [ 4.01723287  1.35771031 -0.13891728  0.86917853  0.73424786  0.61657389
  0.16093621  1.36591271  2.160564    2.07319099  0.18773214 -1.3398344
  0.95151931 -0.17718419  1.30058278 -0.68055017 -0.52425991 -1.12672605
 -2.37678442 -1.50609805 -1.00909109 -1.19981588  0.92901889  0.889823

eval_count: 53
var_params: [ 3.73608899  1.36208443 -0.13954705  0.86684873  0.73452832  0.61495985
  0.159271    1.36643906  2.4066179   2.07369148  0.23209943 -1.34536041
  0.95067438 -0.18168299  1.30258742 -0.68651792 -0.52596265 -1.12707493
 -2.37927095 -1.51196142 -1.01533251 -1.20188154  0.92698829  0.88583697
 -0.99110485 -0.11403539  0.62899806 -1.01961854 -0.78300492 -2.37874183
 -0.3641606  -0.60215336  0.39230672  0.15480174 -0.91709694 -0.36725079
  1.06805666 -0.78025887]
eval_val: 0.5548815584843448
index: 53
eval_count: 54
var_params: [ 3.75542687  1.35240928 -0.14332717  0.8610723   0.71293683  0.59009918
  0.14430246  1.37235337  2.05378437  2.07688601  0.66207986 -1.37382192
  0.94528139 -0.2047783   1.30150955 -0.71073575 -0.52295779 -1.11542871
 -2.38126886 -1.53551299 -1.04129707 -1.2011931   0.92790048  0.8742622
 -1.03142296 -0.11944557  0.63222913 -1.03885018 -0.78892081 -2.41117541
 -0.38049815 -0.60401927  0.3892907   0.15201601 -0.9136275  -0.37440017
  1.03

...

eval_count: 197
var_params: [ 3.50242914  1.20617958 -0.16130817  0.7469649   0.84477051  0.21873698
  0.21068853  1.66846445  1.5340526   1.86822892  0.1789352  -1.07541506
  1.06638703 -0.63632295  1.05751314 -1.06319923 -0.66218542 -1.12066031
 -2.45767655 -1.49805221 -0.54476576 -0.71625317  0.75153496  0.88513353
 -1.06615514 -0.22947298  1.13634736 -0.75108013 -0.50148656 -2.04177209
  0.26858488 -0.59654554  0.54041181 -0.28284235 -0.11503815 -0.05630691
  1.64307975 -0.25577639]
eval_val: 0.4785150556148832
index: 197
eval_count: 198
var_params: [ 3.45497222  1.18603978 -0.13712824  0.75013602  0.8138562   0.26876865
  0.24029613  1.67633894  1.53265973  1.85897326  0.1751643  -1.07899957
  1.06097184 -0.64406804  1.01840852 -1.07102266 -0.57427788 -1.16470492
 -2.33943482 -1.48655092 -0.52681523 -0.76361598  0.79121372  0.82090939
 -1.05784073 -0.22863818  1.1023799  -0.75539993 -0.52736638 -2.04475128
  0.2873293  -0.73171674  0.50644656 -0.26231558 -0.02448472  0.02169657
  

eval_count: 213
var_params: [ 3.38667904  1.21119127 -0.16564415  0.69517371  0.88549905  0.322451
  0.17575714  1.69514331  1.54728246  1.76109222  0.11091575 -0.98366765
  1.01122889 -0.58881951  1.07978738 -0.92002571 -0.52412297 -1.20937681
 -2.26855554 -1.60264074 -0.58267052 -0.80011552  0.80156732  0.87726609
 -1.08746027 -0.28316118  0.99860541 -0.84697305 -0.57665378 -2.11835347
  0.24497527 -0.72090479  0.54643214 -0.27456968 -0.08223386  0.04750985
  1.55621031 -0.34036948]
eval_val: 0.4773771541752027
index: 213
eval_count: 214
var_params: [ 3.48809606  1.23640323 -0.19056068  0.77158784  0.82266683  0.26430623
  0.16343844  1.69351087  1.52703154  1.85395125  0.14044315 -1.06026601
  1.09267567 -0.66111904  1.0247279  -0.94972414 -0.52803922 -1.25867875
 -2.30655063 -1.56707828 -0.59102231 -0.78289574  0.80329258  0.87012394
 -1.07116908 -0.22420811  1.01536692 -0.8388464  -0.54521384 -2.14252025
  0.23021247 -0.69628343  0.47357085 -0.29007662 -0.04036145  0.0756296
  1.5

eval_count: 229
var_params: [ 3.48135923  1.16813506 -0.23188062  0.74778775  0.8582752   0.50782514
  0.320715    1.77057882  1.50987048  2.0835484   0.11496738 -0.99033969
  1.1780231  -0.73306897  1.15069668 -0.85717217 -0.66294527 -1.2624287
 -2.32755914 -1.49563387 -0.45572159 -0.83813549  0.79488093  0.90746896
 -1.00848108 -0.24588161  0.99465739 -0.88208969 -0.45165036 -1.99829204
  0.22334354 -0.79553017  0.55397037 -0.33994848 -0.00388838  0.0543783
  1.49179055 -0.47409473]
eval_val: 0.47516023204944374
index: 229
eval_count: 230
var_params: [ 3.51403133  1.11198912 -0.25405456  0.75558735  0.91366117  0.49321263
  0.3070327   1.70109039  1.53100368  2.07669326  0.09186262 -1.01098063
  1.15069182 -0.67991938  1.17639603 -0.88936051 -0.60097471 -1.23285604
 -2.29608582 -1.48606996 -0.44930914 -0.8749463   0.78213253  0.9145343
 -1.09874388 -0.22982766  1.03381721 -0.85022073 -0.55970004 -2.05026562
  0.24901602 -0.72756989  0.49716704 -0.37379223 -0.03949538  0.11173245
  1.

eval_count: 245
var_params: [ 3.54181477  1.19577072 -0.27348214  0.75911613  0.95732906  0.43581132
  0.32971884  1.75123672  1.44407228  2.08775668  0.09181987 -0.98158848
  1.16292235 -0.59234598  1.17369756 -0.88095179 -0.6108126  -1.19720067
 -2.3056002  -1.48055133 -0.44267394 -0.92275224  0.77741182  0.8291134
 -1.11248627 -0.23226888  0.9755672  -0.89896825 -0.47518192 -1.96864285
  0.16774868 -0.73516546  0.4993913  -0.34444016  0.01152654  0.22841068
  1.66357382 -0.38527308]
eval_val: 0.47155982299864774
index: 245
eval_count: 246
var_params: [ 3.54396744  1.1317323  -0.22790094  0.74759525  0.93103289  0.46022303
  0.27560219  1.77071433  1.52484777  2.01542015  0.0440169  -0.99323227
  1.24011014 -0.66836729  1.23310959 -0.8862035  -0.61425466 -1.2179175
 -2.28622238 -1.49687222 -0.47673217 -0.86645798  0.76974292  0.84563763
 -1.05292198 -0.26770146  1.027669   -0.88074197 -0.53287594 -2.00503443
  0.23163815 -0.72751937  0.54433775 -0.35505218 -0.00536888  0.14059265
  1

eval_count: 261
var_params: [ 3.63735488  1.23010098 -0.19727682  0.75518914  0.93623032  0.33866258
  0.15493558  1.79454389  1.53789262  2.1126545   0.06563415 -0.985916
  1.11019378 -0.73087079  1.20382466 -0.96182866 -0.60713472 -1.41930655
 -2.15243637 -1.50236104 -0.44846034 -0.88797509  0.82534014  0.81314946
 -1.05324473 -0.12395106  1.07409928 -0.80064429 -0.50141729 -2.08496815
  0.17098267 -0.6632748   0.55194929 -0.37890751  0.00707168  0.14470199
  1.59440104 -0.40903207]
eval_val: 0.47184477202917335
index: 261
eval_count: 262
var_params: [ 3.65608744  1.23416509 -0.30621785  0.73795676  0.96491525  0.37806172
  0.21464241  1.79811962  1.5757282   2.12412887  0.15536011 -0.98683179
  1.18179945 -0.69739371  1.18831452 -0.9790955  -0.63101585 -1.41774123
 -2.24300828 -1.52434858 -0.43408939 -0.86361421  0.80556391  0.81486084
 -1.07113078 -0.23698896  1.07991837 -0.88803046 -0.52606246 -2.08371612
  0.25659305 -0.7005138   0.5508416  -0.35426764  0.029265    0.15823399
  1

## Storing the optimal parameters for grading

Once the training step of the vqc algorithm is done we obtain the optimal parameters for our specific variational form. For the grading function to be able to access these optimal parameters you will need to follow the steps below. 

* **Step 1**: Run the cell below with `print(repr(vqc.optimal_params))`. 
* **Step 2**: Copy the matrix of optimal parameters and store it in the variable `optimal_parameters` inside the function `return_optimal_params()` in the next cell. This will enable us to extract it while calculating the accuracy your the model during grading. Given below is a pictographical explanation of the same:  

<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/opt_params456b075.png" width="800">

In [157]:
print(repr(vqc.optimal_params))

array([-1.67142601e+00,  5.93502332e-01,  2.12410944e+00,  1.29322495e+00,
        1.58951842e+00,  1.16688159e+00,  1.52028676e+00,  1.65337475e+00,
        9.48271996e-01,  7.12327424e-01, -1.45868725e+00, -1.01304819e+00,
        1.89530752e+00, -9.45549297e-02,  2.03751127e+00,  1.16335652e+00,
       -1.31152393e+00, -4.33788054e-01,  2.03344822e-01,  7.52335867e-01,
       -1.11387158e+00,  3.78823964e-01, -1.35112329e-01,  1.08083482e-01,
        1.60209807e+00,  1.11522547e+00, -1.10481258e+00, -9.77877169e-01,
        4.67243779e-02,  1.51123664e+00,  1.06101871e+00, -1.42398700e-03,
       -1.50699343e+00, -9.70594149e-01, -1.29007891e-01,  9.02937147e-01,
        1.15025025e+00,  6.60572232e-01,  4.49962801e-01,  1.56709013e+00])


In [38]:
%%write_and_run optimal_params.py
# # the write_and_run function writes the content in this cell into the file "optimal_params.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
import numpy as np
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def return_optimal_params():
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    
    optimal_parameters =[ 3.63795705 , 1.19128865 ,-0.33132838 , 0.74721243 , 0.92831932 , 0.42240372,
  0.17701117 , 1.80778497,  1.54819545 , 2.14835959 , 0.0773745 , -1.0679474,
  1.10294989 ,-0.69464833,  1.17269839 ,-1.03707315 ,-0.64726175 ,-1.42869361,
 -2.25336805 ,-1.54722281 ,-0.5098562 , -0.83212129 , 0.8265452  , 0.78973385,
 -1.01799251, -0.18191186 , 1.05282282, -0.93642473 ,-0.60919723 ,-2.02198214,
  0.25290384 ,-0.69921034  ,0.52687956, -0.26904783 , 0.01643734 , 0.17502405,
  1.64029289, -0.43707053]
    
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    return np.array(optimal_parameters)

## Submission

Before we go any further, check that you have the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` in the **same working directory as this notebook**. If you do not, then go back to the start and run the notebook making sure you have filled in the code where its required. When you run the cell below, all the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` are combined into one file named **"answer.py"**. Now your working directory will have four python (.py) files out of which **"answer.py"** is the submission file: 
* `answer.py` <- upload this file onto HackerEarth and click on "Submit and Evaluate"
* `feature_map.py`
* `variational_circuit.py`
* `optimal_params.py`

In [39]:
solution = ['feature_map.py','variational_circuit.py','optimal_params.py']
file = open("answer.py","w")
file.truncate(0)
for i in solution:    
    with open(i) as f:
        with open("answer.py", "a") as f1:
            for line in f:
                f1.write(line)
file.close()

## Grading Function

Given below is the grading function that we shall use to grade your submission with a test dataset that is of the same format as `challenge_dataset_4_9.csv`. You can use it to grade your submission by extracting a few points out of the `challenge_dataset_4_9.csv` to get a basic idea of how your model is performing. 

In [40]:
#imports required for the grading function 
from qiskit import *
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.feature_maps import FeatureMap
from qiskit.aqua.components.variational_forms import VariationalForm
import numpy as np

### Working of the grading function

The grading function `grade()` takes as **input**: 

* `test_data`: (`np.ndarray`) -- **no. of datapoints $\times$ dimension of data** : the datapoints against which we want to test our model. 


* `test_labels`: (`np.ndarray`) -- **no. of datapoints $\times$ 1** : A column vector with each entry either 0 or 1 as entries.


* `feature_map`: (`QuantumCircuit` or `FeatureMap`) -- A quantum feature map which is the output of `feature_map()` defined earlier.


* `variational_form`: (`QuantumCircuit` or `VariationalForm`) -- A variational form which is the output of `variational_circuit()` defined earlier.


* `optimal_params`: (`numpy.ndarray`) -- the optimal parameters obtained after running the VQC algorithm above. These are the values obtained when the function `return_optimal_params()` is run. 


* `find_circuit_cost` : (`bool`) -- Calculates the circuit cost if set to `True`. Circuit cost is calculated by converting the circuit to the basis gate set `\[ 'u3', 'cx'\]` and then applying the formula **cost = 1$\times$(no.of u3 gates) + 10$\times$(no.of cx gates)**.


* `verbose` : (`bool`) -- prints the result message if set to `True`.

And gives as **output**: 

* `model_accuracy` : (`numpy.float64`) -- percent accuracy of the model. 


* `circuit_cost`: (`int`) -- circuit cost as explained above.


* `ans`: (`tuple`) -- Output of the `VQC.predict()` method. 


* `result_msg`: (`str`) -- Result message which also outputs the error message in case of one.


* `unrolled_circuit`: (`QuantumCircuit` or `None`) -- the circuit obtained after unrolling the full VQC circuit and substituting the optimal parameters to the basis gate set `\[ 'u3', 'cx'\]`.

**Note:** if you look inside the `grade()` function in Section 2 you'll see that we have initialized a COBYLA optimizer though the prediction step will not require one. Similarily we have given a dataset to `training dataset`. Both of these are dummy variables. The reason for this is because these are not optional variables the `VQC` class instantiation.  

In [41]:
def grade(test_data, test_labels, feature_map, variational_form, optimal_params, find_circuit_cost=True, verbose=True):
    seed = 10598
    model_accuracy = None 
    circuit_cost=None 
    ans = None
    unrolled_circuit = None
    result_msg=''
    data_dim = np.array(test_data).shape[1]
    dataset_size = np.array(test_data).shape[0]
    dummy_training_dataset=training_input = {'A':np.ones((2,data_dim)), 'B':np.ones((2, data_dim))}
    
    # converting 4's to 0's and 9's to 1's for checking 
    test_labels_transformed = np.where(test_labels==4, 0., 1.)
    max_qubit_count = 6
    max_circuit_cost = 2000
    
    # Section 1
    if feature_map is None:
        result_msg += 'feature_map variable is None. Please submit a valid entry' if verbose else ''
    elif variational_form is None: 
        result_msg += 'variational_form variable is None. Please submit a valid entry' if verbose else ''
    elif optimal_params is None: 
        result_msg += 'optimal_params variable is None. Please submit a valid entry' if verbose else ''
    elif test_data is None: 
        result_msg += 'test_data variable is None. Please submit a valid entry' if verbose else ''
    elif test_labels is None: 
        result_msg += 'test_labels variable is None. Please submit a valid entry' if verbose else ''
    elif not isinstance(feature_map, (QuantumCircuit, FeatureMap)):
        result_msg += 'feature_map variable should be a QuantumCircuit or a FeatureMap not (%s)' % \
                      type(feature_map) if verbose else ''
    elif not isinstance(variational_form, (QuantumCircuit, VariationalForm)):
        result_msg += 'variational_form variable should be a QuantumCircuit or a VariationalForm not (%s)' % \
                      type(variational_form) if verbose else ''
    elif not isinstance(test_data, np.ndarray):
        result_msg += 'test_data variable should be a numpy.ndarray not (%s)' % \
                      type(test_data) if verbose else ''
    elif not isinstance(test_labels, np.ndarray):
        result_msg += 'test_labels variable should be a numpy.ndarray not (%s)' % \
                      type(test_labels) if verbose else ''
    elif not isinstance(optimal_params, np.ndarray):
        result_msg += 'optimal_params variable should be a numpy.ndarray not (%s)' % \
                      type(optimal_params) if verbose else ''
    elif not dataset_size == test_labels_transformed.shape[0]:
        result_msg += 'Dataset size and label array size must be equal'
    # Section 2
    else:
        
        # setting up COBYLA optimizer as a dummy optimizer
        from qiskit.aqua.components.optimizers import COBYLA
        dummy_optimizer = COBYLA()

        # setting up the backend and creating a quantum instance
        backend = Aer.get_backend('qasm_simulator')
        backend_options = {"method": "statevector"}
        quantum_instance = QuantumInstance(backend, 
                                           shots=2000, 
                                           seed_simulator=seed, 
                                           seed_transpiler=seed, 
                                           backend_options=backend_options)

        # creating a VQC instance and running the VQC.predict method to get the accuracy of the model 
        vqc = VQC(optimizer=dummy_optimizer, 
                  feature_map=feature_map, 
                  var_form=variational_form, 
                  training_dataset=dummy_training_dataset)
        
        from qiskit.transpiler import PassManager
        from qiskit.transpiler.passes import Unroller
        pass_ = Unroller(['u3', 'cx'])
        pm = PassManager(pass_)
        # construct circuit with first datapoint
        circuit = vqc.construct_circuit(data[0], optimal_params)
        unrolled_circuit = pm.run(circuit)
        gates = unrolled_circuit.count_ops()
        if 'u3' in gates: 
            circuit_cost = gates['u3']
        if 'cx' in gates: 
            circuit_cost+= 10*gates['cx']
        
        if circuit.num_qubits > max_qubit_count:
            result_msg += 'Your quantum circuit is using more than 6 qubits. Reduce the number of qubits used and try again.'
        elif circuit_cost > max_circuit_cost:
            result_msg += 'The cost of your circuit is exceeding the maximum accpetable cost of 2000. Reduce the circuit cost and try again.'
        else: 
            
            ans = vqc.predict(test_data, quantum_instance=quantum_instance, params=np.array(optimal_params))
            model_accuracy = np.sum(np.equal(test_labels_transformed, ans[1]))/len(ans[1])

            result_msg += 'Accuracy of the model is {}'.format(model_accuracy) if verbose else ''
            result_msg += ' and circuit cost is {}'.format(circuit_cost) if verbose else ''
            
    return model_accuracy, circuit_cost, ans, result_msg, unrolled_circuit

## Process of grading using a dummy grading dataset

Let us create a dummy grading dataset with features and labels `grading_features` and `grading_labels` created from the last 2000 datapoints from `data_features` and `data_labels`so that we can a rough estimate of our accuaracy. It must be noted that this may not be a balanced dataset, i.e, may not have equal number of `4`'s and `9`'s and is not best practice. This is only given for the purpose of the demo of `grade()` function. In the final scoring done on HackerEarth, the testing dataset used will have a balanced number of class labels `4` and `9`.

In [42]:
grading_dataset_size=2000    # this value is not per digit but in total
grading_features = data_features[-grading_dataset_size:]
grading_labels = data_labels[-grading_dataset_size:]

In [43]:
start = time.process_time()

accuracy, circuit_cost, ans, result_msg, full_circuit  =  grade(test_data=grading_features, 
                                                                test_labels=grading_labels, 
                                                                feature_map=feature_map(), 
                                                                variational_form=variational_circuit(), 
                                                                optimal_params=return_optimal_params())

print("time taken: {} seconds".format(time.process_time() - start))
print(result_msg)

c:\users\codie\appdata\local\programs\python\python37\lib\site-packages\qiskit\aqua\algorithms\classifiers\vqc.py:146: DeprecationWarning:

The qiskit.aqua.components.feature_maps.FeatureMap object is deprecated as of 0.7.0 and will be removed no earlier than 3 months after the release. You should pass a QuantumCircuit object instead. See also qiskit.circuit.library.data_preparation for a collection of suitable circuits.



time taken: 120.921875 seconds
Accuracy of the model is 0.7885 and circuit cost is 612


You can also check your **accuracy**, **circuit_cost** and **full_circuit** which is the result of combining the feature map and variational circuit and unrolling into the basis \['u3', 'cx'\].

In [ ]:
print("Accuracy of the model: {}".format(accuracy))
print("Circuit Cost: {}".format(circuit_cost))
print("The complete unrolled circuit: ")
full_circuit.draw()